<a href="https://colab.research.google.com/github/AchrafAsh/awesome-pytorch-notebooks/blob/main/01_machine_translation_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation with Transformers

In [ ]:
from typing import List

import torch
import torch.nn as nn
import torch.nn.functional as F

## Vocabulary

In [ ]:
class Vocab:
    def __init__(self):
        self.itos = {0: "<PAD>", 1: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<UNK>": 1}

    def build_vocab(self, sentences: List[str]):
        idx = 0
        for sentence in sentences:
            # TODO: tokenize the sentence
            # TODO: add the word in itos and stoi if not already
            # TODO: increment the index
        print(f"Done ✅. Added {idx} words.")
    
    def encode(self, sentence: str):
        tokens = []
        for w in sentence:
            if w not in self.stoi:
                tokens.append(4) # unknown word
            else:
                tokens.append(self.stoi[w])
        
        return tokens
    
    def decode(self, tensor: torch.tensor):
        pass

## Architecture

- Embeddings → encode one-hot-encoded words as continuous vectors to catch semantic (might use pre-trained word2vec for that)
- Transformer Block stacked

In [ ]:
class Translator(nn.Module):
    def __init__(self, src_vocab_size, trgt_vocab_size):
    
    def forward(self, x)